## Final Project Submission

Please fill out:
* Student name: Dara Estrada
* Student pace: Full time
* Scheduled project review date/time: Jan 28, 2022
* Instructor name: Angelica, Diane
* Blog post URL:


In [ ]:
import pandas as pd
import gzip as gz
import matplotlib.pyplot as plt
import sqlite3
import os
import zipfile
import seaborn as sns
import math
import random
from sklearn.linear_model import LinearRegression

In [ ]:
# Opened the relevant datasets to be explored in jupyter notebook
#if not os.path.exists('zippedData/im.db'):
with zipfile.ZipFile('zippedData/im.db.zip') as my_zip:
    zipfile.ZipFile.extractall(my_zip,path='zippedData/')
with gz.open('zippedData/rt.reviews.tsv.gz') as f:
    rt_reviews = pd.read_csv(f,delimiter='\t',encoding='latin1')
with gz.open('zippedData/rt.movie_info.tsv.gz') as f:
    rt_movie_info = pd.read_csv(f,delimiter='\t',encoding='latin1')
with gz.open('zippedData/tmdb.movies.csv.gz') as f:
    tmdb_movies = pd.read_csv(f)
with gz.open('zippedData/tn.movie_budgets.csv.gz') as f:
    tn_movie_budgets = pd.read_csv(f)
with gz.open('zippedData/bom.movie_gross.csv.gz') as f:
    bom_movie_gross = pd.read_csv(f)

In [ ]:
# Connected to and opened the imdb database to utilize the data
im_db = sqlite3.connect('zippedData/im.db')
imdb_df = pd.read_sql("""SELECT * FROM movie_basics""",im_db)

In [ ]:
# Created a SQL query to pull director name and info from the imdb database
director_info = """
SELECT
    m.movie_id,
    m.primary_title,
    m.genres,
    d.person_id,
    p.primary_name,
    p.death_year
    
    
FROM
    movie_basics AS m
    JOIN 
        directors AS d ON m.movie_id = d.movie_id
    JOIN
        known_for AS kf ON m.movie_id = kf.movie_id
    JOIN
        persons AS p ON p.person_id = d.person_id
GROUP BY
    m.movie_id, kf.person_id, p.primary_name
;
"""
director_info_result = pd.read_sql(director_info, im_db)
director_info_result

In [ ]:
tn_movie_budgets['year'] = tn_movie_budgets['release_date'].str[-4:]
tn_movie_budgets['month'] = tn_movie_budgets['release_date'].str[:3]
tn_movie_budgets['clean_budget'] = tn_movie_budgets['production_budget'].str.replace('$','')
tn_movie_budgets['clean_budget'] = tn_movie_budgets['clean_budget'].str.replace(',','').astype(int)
tn_movie_budgets['clean_domestic'] = tn_movie_budgets['domestic_gross'].str.replace('$','')
tn_movie_budgets['clean_domestic'] = tn_movie_budgets['clean_domestic'].str.replace(',','').astype(int)
tn_movie_budgets['clean_worldwide'] = tn_movie_budgets['worldwide_gross'].str.replace('$','')
tn_movie_budgets['clean_worldwide'] = tn_movie_budgets['clean_worldwide'].str.replace(',','').astype('int64')
tn_movie_budgets['clean_foreign'] = tn_movie_budgets['clean_worldwide'] - tn_movie_budgets['clean_domestic']
tn_movie_budgets.rename({'movie':'title'},axis=1,inplace=True)
#tn_movie_budgets.info()
tn_relevant_cols = ['title','year','month','clean_budget','clean_domestic','clean_foreign','clean_worldwide']
tn_relevant = tn_movie_budgets[tn_relevant_cols]
profit = tn_relevant['clean_worldwide']- tn_relevant['clean_budget']
tn_relevant['profit'] = profit
tn_relevant.head()

In [ ]:
def season_category(month):
    if month in ['Dec', 'Jan', 'Feb']:
        return 'Winter'
    elif month in ['Mar', 'Apr', 'May']:
        return  'Spring'
    elif month in ['Jun', 'Jul', 'Aug']:
        return 'Summer'
    else:
        return 'Autumn'

In [ ]:
print(season_category('Aug'))

In [ ]:
tn_relevant['Seasons'] = tn_relevant['month'].apply(season_category)

In [ ]:
tn_relevant.head()

In [ ]:
from pylab import *
from numpy import outer
rc('text', usetex=False)
a=outer(arange(0,1,0.01),ones(10))
figure(figsize=(10,5))
subplots_adjust(top=0.8,bottom=0.05,left=0.01,right=0.99)
maps=[m for m in cm.datad if not m.endswith("_r")]
maps.sort()
l=len(maps)+1
for i, m in enumerate(maps):
     subplot(1,l,i+1)
     axis("off")
     imshow(a,aspect='auto',cmap=get_cmap(m),origin="lower")
     title(m,rotation=90,fontsize=10)
savefig("colormaps.png",dpi=100,facecolor='gray')


In [ ]:
ax3=tn_relevant.groupby('Seasons')['clean_worldwide'].median().plot(kind='bar', stacked=True, colormap='summer')

ax3.set_ylabel('Median Worldwide Gorss (Millions)');
ax3.set_yticklabels(ax3.get_yticks()//1000000);
ax3.set_title('Median Earning Per Season')


In [ ]:
ax3=tn_relevant.groupby('month')['clean_worldwide'].median().plot(kind='bar', stacked=True, colormap='summer')

ax3.set_ylabel('Median Worldwide Gorss (Millions)');
ax3.set_yticklabels(ax3.get_yticks()//1000000);
ax3.set_title('Median Earning Per Season')

In [ ]:
months_in_order = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
tn_plot_month = tn_relevant.groupby('month').agg(median_earning=('clean_worldwide','median'))
tn_plot_month= tn_plot_month.reset_index()
sns.set(rc = {'figure.figsize':(15,8)});
sns.set_theme(context = 'talk',style='white');
ax4 = sns.barplot(x='month',y='median_earning',data=tn_plot_month,order=months_in_order)
ax4.set_ylabel('Median Worldwide Gross (Millions)');
ax4.set_xlabel(None);
ax4.set_title('',fontsize=32);
ax4.set_yticklabels(ax4.get_yticks()//1000000);

In [ ]:
colors = ["#348781","#EE9A4D","#931314","#4863A0" ]
sns.set_palette(sns.color_palette(colors))
seasons_in_order = ['Spring','Summer','Autumn','Winter']
tn_plot_season = tn_relevant.groupby('Seasons').agg(median_earning=('clean_worldwide','median'))
tn_plot_season= tn_plot_season.reset_index()
sns.set(rc = {'figure.figsize':(15,8)});
sns.set_theme(context = 'talk',style='white');
sns.set_palette(sns.color_palette(colors));
ax4 = sns.barplot(x='Seasons',y='median_earning',data=tn_plot_season,order=seasons_in_order)
ax4.set_ylabel('Median Worldwide Gross (Millions)');
ax4.set_xlabel(None);
# ax4.set_title('',fontsize=32);
ax4.set_yticklabels(ax4.get_yticks()//1000000);